<a href="https://colab.research.google.com/github/ikramMc/PFE/blob/main/LLM_infrence_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pip3-autoremove
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu124
!pip install unsloth
# !pip install --upgrade transformers==4.52.3

In [ ]:
%%capture
!pip install flask

In [ ]:
import torch
from unsloth import FastLanguageModel
import os
# Charger ton modèle avec un chemin vers ton modèle
model_path ="kimxxxx/mistral_r64_a128_g8_gas8_lr9e-5_4500tk_droplast_nopacking_2epoch"
#"unsloth/mistral-7b-instruct-v0.3-bnb-4bit"#"unsloth/Llama-3.1-8B-unsloth-bnb-4bit"#"meta-llama/Llama-3.1-8B-Instruct"#"kimxxxx/mistral_r64_alpah128_batch8_gradient4_Ler9e-5_withoutsyntctf_ctfman_2epoch" #"kimxxxx/mistral_r64_alpah128_batch16_gradient2_Ler9e-5_fulldataset_2epoch"  # Remplace par ton modèle ou ton repo HuggingFace
token=os.getenv("HF_TOKEN")

# Charger le modèle et le tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=32000,
    dtype=torch.float16,
    load_in_4bit=True,
    token=token,

)

# Préparer le modèle pour l'inférence
FastLanguageModel.for_inference(model)


In [ ]:

import torch
import gc

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

# Initialize Flask
app = Flask(__name__)

@app.route('/generate', methods=['POST'])
def generate():
    user_input = request.json.get('input', '')
    print("user prompt")
    print(user_input)
    print("**********************************")
    prompt = [{"role": "user", "content": user_input}]

    formatted_prompt = tokenizer.apply_chat_template(prompt, tokenize=False) if hasattr(tokenizer, "apply_chat_template") else user_input

    try:
        with torch.no_grad():
            # Tokenize and move to device
            inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)
            outputs = model.generate(
                **inputs,
                max_new_tokens=400,
                do_sample=True,
                temperature=0.7,
                top_p=0.9
            )
            input_length = inputs['input_ids'].shape[1]  # length of prompt tokens
            generated_tokens = outputs[0][input_length:]  # slice off prompt tokens
            generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

        # Explicitly clear memory after generation
        del inputs
        del outputs
        torch.cuda.empty_cache()
        gc.collect()

        print(generated_text)
        print("*********************************")
        return jsonify({"response": generated_text})

    except Exception as e:
        # Also try to clear cache in case of exceptions
        torch.cuda.empty_cache()
        gc.collect()
        return jsonify({"error": str(e)}), 500

# Expose the API with ngrok
public_url = ngrok.connect(5000)  # Ensure this is the same port your Flask app uses
print(f"ngrok tunnel \"{public_url}\" -> \"http://localhost:5000\"")

app.run()
